In [36]:
import numpy as np
import pandas as pd

import pandas as pd
import numpy as np
from datetime import timedelta, date
from tqdm import tqdm

In [37]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
start_date = date(2020, 2, 28)
end_date = date(2020, 4, 30)

In [38]:
flow_matrices = []

for date_id, single_date in tqdm(enumerate(daterange(start_date, end_date))):
    date_ = single_date.strftime("%Y-%m-%d")
    filename = '../COVIDExposureIndices/lex_data/county_lex_%s.csv.gz' % date_
    df = pd.read_csv(filename)
    fips = np.array(df.COUNTY_PRE)
    flow_matrices.append(np.array(df)[:, 1:])

62it [00:36,  1.70it/s]


In [120]:
flow_feats = np.stack(flow_matrices)

In [406]:
case_count_f = '../data/county_case_counts_time_series_data_%s.csv'
mobility_f = '../data/mobility_%s.csv'
unemploy_f = '../data/temporal_%s_2.csv'
census_f = '../data/2019_county_level_population_data.csv'
census65_f = '../data/population_with_age_gte_65.csv'
location_f = '../data/county_locations.csv'

In [407]:
subset = 'train'
# temporal features
df_count = pd.read_csv(case_count_f %subset, index_col=0)
df_mobility = pd.read_csv(mobility_f %subset, index_col=0)
df_unemploy = pd.read_csv(unemploy_f %subset, index_col=0)

# static features
df_census = pd.read_csv(census_f)
df_census.us_census_population_2019 = df_census.us_census_population_2019.apply(lambda s: s.replace(',', ''))
df_census65 = pd.read_csv(census65_f)
df_census = df_census65.merge(df_census, on='FIPS')

# location features
df_location = pd.read_csv(location_f)

In [558]:
np.where(fips==13215)

(array([350]),)

In [559]:
fips[344]

13195

In [402]:
fips_mobility = df_mobility.county_fips.unique()
fips_loc = df_location.FIPS.unique()
fips_census = df_census.FIPS.unique()
fips_count = df_count.fips.unique()

fips_census = df_census.FIPS.unique()
fips


array([ 1001,  1003,  1005, ..., 56037, 56039, 56041])

In [ ]:
df_une

In [403]:
# geo_features: [num_counties, 2]
# static_node_features: [num_counties, num_features]
# node_features: [num_counties, num_days, num_features]
# edge_features: [num_counties, num_counties, num_days]

In [428]:
df_unemploy[(df_unemploy.county_fips==county) & (df_unemploy.Month==month)].unemploymentRate

1    2.790788
Name: unemploymentRate, dtype: float64

In [442]:
# merge temp features
fips_count.sort()
y = []
temp_node_feats = []
static_node_feats = []
geo_feats = []
for county in tqdm(fips_count):
    y.append(np.array(df_count[df_count.fips==county].case_count_rolling_average))
    # death_counts = np.array(df_count[df_count.fips==county].death_count)
    mobility_features = np.array(df_mobility[df_mobility.county_fips==county].fillna(0).iloc[:, 2:8])
    
    unemployment_features = []
    for date_id, single_date in enumerate(daterange(start_date, end_date)):
        month = single_date.month
        try:
            unemployment_features.append(np.array(df_unemploy[(df_unemploy.county_fips==county) & (df_unemploy.Month==month)].unemploymentRate)[0])
        except:
            unemployment_features.append(0.)
            print(county, end=' ')
    unemployment_features = np.array(unemployment_features)[:, np.newaxis]
    temp_feats = np.concatenate([mobility_features, unemployment_features], axis=-1)
    # add unemployement
    temp_node_feats.append(temp_feats)
    
    static_node_feats.append(np.array(df_census[df_census.FIPS==county][['us_census_population_2019', 'pop_age_gte_65']]))
    
    geo_feats.append(np.array(df_location[df_location.FIPS==county].iloc[0, 3:]))

 20%|██        | 601/2942 [00:29<01:54, 20.43it/s]

17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 17073 

 44%|████▎     | 1282/2942 [01:02<01:18, 21.16it/s]

27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 27075 

 87%|████████▋ | 2558/2942 [02:03<00:18, 21.18it/s]

48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 48423 

 93%|█████████▎| 2729/2942 [02:11<00:10, 20.90it/s]

51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 51570 

100%|██████████| 2942/2942 [02:21<00:00, 20.74it/s]


In [449]:
train_data = {'y': y, # num_counties, num_days
             'edge_matrix': day_edge_matrix,  # num_days, num_counties, num_counties
             #'geo_feat': geo_feats, # num_couties, 2
             'static_node_feats': static_node_feats, # num_counties, num_features
             'temp_node_feats': temp_node_feats,} # num_counties, num_days, num_features

In [450]:
np.save("train.npy", train_data)

In [443]:
y = np.stack(y) # num_counties, num_days
temp_node_feats = np.stack(temp_node_feats) # num_counties, num_days, num_features
static_node_feats = np.stack(static_node_feats).squeeze() # num_counties, num_features
geo_feats = np.stack(geo_feats).astype(np.float) # num_counties, 2

In [447]:
temp_node_feats.shape

(2942, 62, 7)

In [227]:
def broadcasting_based_lng_lat(data1, data2):
    # data1, data2 are the data arrays with 2 cols and they hold
    # lat., lng. values in those cols respectively
    data1 = np.deg2rad(data1)                     
    data2 = np.deg2rad(data2)                     

    lat1 = data1[:,0]                     
    lng1 = data1[:,1]         

    lat2 = data2[:,0]                     
    lng2 = data2[:,1]         

    diff_lat = lat1[:,None] - lat2
    diff_lng = lng1[:,None] - lng2
    d = np.sin(diff_lat/2)**2 + np.cos(lat1[:,None])*np.cos(lat2) * np.sin(diff_lng/2)**2
    return 2 * 6371 * np.arcsin(np.sqrt(d))

In [231]:
dist_matrix = broadcasting_based_lng_lat(geo_feats, geo_feats)

In [293]:
edge_index = []
for i in range(dist_matrix.shape[0]):
    edge_index.append(dist_matrix[i].argsort()[:16])
edge_index = np.stack(edge_index)

In [379]:
geo_feats.shape

(2942, 2)

In [309]:
edge_matrix = np.zeros(dist_matrix.shape)
for i, topk in enumerate(edge_index):
    edge_matrix[i, topk] = 1

In [375]:
edge_matrix.shape

(2942, 2942)

In [ ]:
day_edge_matrix = []
for day in range(flow_feats.shape[0]):
    edge_matrix_2 = np.zeros(dist_matrix.shape)
    for i, fip in enumerate(fips):
        try:
            cc = np.where(fips[i] == fips_count)[0][0]
            edge_matrix_2[cc][fips_map_j] = flow_feats[day][i, fips_map_i]
        except:
            print(i, end=' ')
            continue
    day_edge_matrix.append(edge_matrix_2)

In [361]:
day_edge_matrix = np.stack(day_edge_matrix)

In [362]:
day_edge_matrix.shape

(62, 2942, 2942)

In [376]:
day_edge_matrix[day_edge_matrix<0.2] = 0
day_edge_matrix += edge_matrix

In [374]:
day_edge_matrix.shape 

(62, 2942, 2942)

In [366]:
fips_map_i = []
fips_map_j = []

for i in range(len(fips)):
    try:
        fips_map_j.append(np.where(fips[i] == fips_count)[0][0])
        fips_map_i.append(i)
    except:
        continue
        
fips_map_i = np.array(fips_map_i)
fips_map_j = np.array(fips_map_j)

In [ ]:
[i in fips_map_j for x in range(500)]

In [313]:
fips_map_j

array([   0,    1,    2, ..., 2937, 2938, 2939])

In [240]:
flow_feats_corrected = -np.ones((flow_feats.shape[0], len(fips_count), len(fips_count)), dtype=flow_feats.dtype)
for day in range(flow_feats.shape[0]):
    flow_feats_corrected[day, fips_map_j, 0:][0:, fips_map_j] = flow_feats[day, fips_map_i, :][:, fips_map_i]
    break

In [243]:
flow_feats_corrected[day].shape

(2942, 2942)

In [ ]:
def replace_submatrix(mat, ind1, ind2, mat_replace):
    for i, index in enumerate(ind1):
        mat[index, ind2] = mat_replace[i, :]
    return mat

In [178]:
replace_submatrix(flow_feats_corrected, )

0.0

In [161]:
flow_feats_corrected[0, fips_map_j, :][0, :, fips_map_j]

IndexError: too many indices for array

In [170]:
flow_feats_corrected[:, fips_map_j, :][:, :, fips_map_j].shape

(62, 1984, 1984)